### Goal: extract data from the documents

1. High level doc reading

In [1]:
from llmsherpa.readers import LayoutPDFReader

# extract pdf dource files and read them as 
def read_file_layout(file_name):
    llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
    pdf_reader = LayoutPDFReader(llmsherpa_api_url)
    doc = pdf_reader.read_pdf(file_name)
    return doc

{
    file_title:...,
    summary:....,
    table of content:.....,
    source:....., 

}

In [3]:
from pydantic import BaseModel, Field
from typing import List,Optional 


class DocChunk(BaseModel):
    chunk_id :  str = Field(description="this is the chunk's id")
    chunk_text : str = Field(description="this is the chunks' text")
    chunk_source : str = Field(description="this is the chunk's source document")
    chunk_doc_location : str = Field(description="this is the path to the chunk in the document")
    chunk_keywords : List[str] = Field(description="this is a list of the chunk's keywords which could be entities or concepts")
class PDFDocDataSource(BaseModel): 
    file_name : str = Field(description="this is the name od the document")
    summary : Optional[str] = Field(default='',description="this is a summary of the documents ")
    table_of_content : Optional[List[str]] = Field(default=[],description="This is a list of the document's sections")
    source : Optional[str] = Field(default='',description="this is the produsing entity of the document")
    chunks : List[DocChunk] =  Field(description="this is a list of the document's chunks")

In [ ]:
def get_file_chunks(doc):
    chunks_as_text = []
    chunks_lst = doc.chunks()
    for chunk in chunks_lst:
        chunk_txt = chunk.to_text()
        chunks_as_text.append(chunk_txt)
    return chunks_as_text

In [6]:
def get_file_sections(doc):
    sections = []
    sections_lst = doc.sections()
    for section in sections_lst:
        section_txt = section.to_text(include_children=True,recurse=True)
        section_title = section.title 
        sections.append( {"section":section_title, "section_text": section_txt})
    return sections

In [ ]:
doc = read_file_layout("/home/asmaa/lablabai-hackathons/hygiene-hero/backend/assets/pdf-docs/Heavy menstrual bleeding diagnosis and medical management.pdf")
sections = get_file_sections(doc)
sections

In [9]:
def get_toc_unstructured(doc_sections):
    toc = []
    for s in doc_sections:
        toc.append(s['section'])
        
    return toc

In [ ]:
import os

def read_docs_data(folder_path):
    docs_data = []
    for filename in os.listdir(folder_path):
        
        if filename.endswith('.pdf'):
            pdf_doc_obj = PDFDocDataSource(file_name=filename)
            file_path = os.path.join(folder_path, filename)
        
        